In [23]:
import gensim.models
import gensim.corpora
import gensim as gs
import pyLDAvis as pvis
import pyLDAvis.gensim
import gensim.models.coherencemodel
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.metrics import f1_score
from gensim.models import FastText
from sklearn.metrics import classification_report

In [37]:
def save_models(dataset_name, num_topics):
    print("loading topic data for", dataset_name)
    # load inputs and labels
    dataset = pd.read_csv("../cleaned/" + dataset_name + "_stems.csv").astype(str).values.tolist() 
    # remove placeholders from the stems dataset
    print("removing placeholders")
    for index, sample in enumerate(dataset): 
            dataset[index] = list(filter((" ").__ne__, sample))
    # create dic, copora and lda-model
    print("making dic")
    dic = gs.corpora.Dictionary(dataset)
    #dic.save("../models/test/" + dataset_name + "_dictionary")
    print("making corpus")
    corpus = [dic.doc2bow(sample) for sample in dataset]
    print("making lda")
    #lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus, id2word=dic, num_topics=num_topics, random_state=100, chunksize=100, passes=10, per_word_topics=True)#update_every=1, 
    #lda_model.save("../models/test/" + dataset_name + "_ldamodel")
    print("making fasttext")
    inputs = [" ".join(sentence) for sentence in dataset]
    vector_model = FastText(size=32, window=3, min_count=1)
    vector_model.build_vocab(inputs)  
    vector_model.train(sentences=inputs, total_examples=len(inputs), total_words=vector_model.corpus_total_words, epochs=10)
    vector_model.save("../models/test/" + dataset_name + "_fasttext")
    
def load_test(dataset_name, num_topics):
    print("loading dic")
    #dic = gs.corpora.Dictionary.load("../models/dictionary/" + dataset_name + "_dictionary")
    print("loading topic model")
    lda_model = gensim.models.ldamulticore.LdaMulticore.load("../models/topic_models/" + dataset_name + "_ldamodel")
    topics = lda_model.show_topics(num_topics = num_topics)
    print(len(topics))
    print(topics)
    print("loading fasttext")
    #vector_model = FastText.load("../models/word_embeddings/" + dataset_name + "_fasttext")
    
    

In [40]:
num_topics_dict = {
    "norm_tweet": 79,
    "norm_emotion": 186,
}

dataset_name = "norm_tweet"
#save_models(dataset_name, num_topics_dict[dataset_name])

load_test(dataset_name, num_topics_dict[dataset_name] + 10)

loading dic
loading topic model
79
[(0, '0.284*"u" + 0.145*"na" + 0.129*"gon" + 0.087*"littl" + 0.036*"manag" + 0.035*"power" + 0.026*"mouth" + 0.023*"outta" + 0.021*"deep" + 0.007*"deserv"'), (1, '0.293*"god" + 0.066*"tv" + 0.055*"un" + 0.047*"forev" + 0.031*"holiday" + 0.029*"usual" + 0.027*"arm" + 0.027*"coupl" + 0.024*"shitti" + 0.020*"cloud"'), (2, '0.132*"e" + 0.113*"ai" + 0.083*"tire" + 0.081*"r" + 0.056*"understand" + 0.055*"ya" + 0.044*"\'" + 0.044*"liter" + 0.036*"forget" + 0.029*"jus"'), (3, '0.264*"play" + 0.140*"eat" + 0.080*"probabl" + 0.051*"mark" + 0.046*"weird" + 0.031*"chick" + 0.028*"player" + 0.020*"min" + 0.012*"egg" + 0.011*"fart"'), (4, '0.267*"tomorrow" + 0.108*"control" + 0.082*"take" + 0.072*"respect" + 0.042*"beat" + 0.026*"9" + 0.024*"danger" + 0.019*"qualiti" + 0.019*"dumb" + 0.017*"glass"'), (5, '0.181*"night" + 0.143*"miss" + 0.099*"run" + 0.067*"head" + 0.050*"listen" + 0.048*"late" + 0.045*"dream" + 0.032*"coffe" + 0.026*"like" + 0.018*"andand"'), (6, '